# 🥭 Show All Accounts

This notebook tries to display information about all Mango margin accounts.

It fetches the data from Solana, parses it, and then prints it.


## How To Use This Page

Theo code should be runnable as-is. Just click the >> button in the toolbar above, and you should see output appear below the code.

In [ ]:
from pyserum.market import Market

from Context import default_context
from Classes import AccountInfo, Group, MarginAccount, OpenOrders, TokenAccount

def show_all_accounts():
    print("Context:", default_context);

    group = Group.load(default_context)
    print("Group:", group)

    markets = list(map(lambda market: Market.load(default_context.client, market.spot), group.markets))
    print("Markets:", markets)

    prices = group.get_prices()
    for price in prices:
        print(f"Price: {price:,.8f}")

    vaults = AccountInfo.load_multiple(default_context, [token.vault for token in group.tokens])
    print("Vaults:", vaults)

    for index, vault in enumerate(vaults):
        token = TokenAccount.parse(vault.data)
        decimals = group.tokens[index].decimals
        amount = token.amount / (10 ** decimals)
        print(f"Vault token amount[{index}]: {amount:,.8f}")

    import time
    start_time = time.time()
    print("Loading margin accounts...")
    margin_accounts = MarginAccount.load_all_for_group(default_context, default_context.program_id, group)
    print(f"Done. Time taken: {time.time() - start_time}")

    print("Loading open orders accounts...")
    # Calling margin_account.load_open_orders() in a loop for each margin account would take a long time.
    # We can do better. Load all the relevant OpenOrders accounts, and then just call each margin account
    # to map its own OpenOrders from the loaded OpenOrders dictionary.
    open_orders = OpenOrders.load_raw_open_orders_accounts(default_context, group)
    open_orders_by_address = {key: value for key, value in [(str(address), open_orders_account) for address, open_orders_account in open_orders]}
    for margin_account in margin_accounts:
        margin_account.install_open_orders_accounts(group, open_orders_by_address)
    print(f"Done. Time taken: {time.time() - start_time}")

    # print(margin_accounts)

# show_all_accounts()
import cProfile
import pstats
cProfile.run("show_all_accounts()", sort=pstats.SortKey.TIME)
